In [1]:
%reload_ext autoreload
%autoreload 2


In [2]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [3]:
import hydra
from hydra.core.global_hydra import GlobalHydra

GlobalHydra.instance().clear()
# setup hydra config global for loading this notebook
hydra.initialize(config_path="configs", version_base=None)
cfg = hydra.compose(config_name="expr")


In [58]:
from main import setup_environment, get_checkpoint_info, is_training_complete

# Setup environment
device = setup_environment(cfg)

# Get checkpoint info
output_dir, checkpoint = get_checkpoint_info(cfg, num_epochs=cfg.training.num_epochs)

# Instantiate everything
bridge = hydra.utils.instantiate(cfg.bridge)
dataset = hydra.utils.instantiate(cfg.dataset, hvg_only=False)
# model = hydra.utils.instantiate(cfg.model)
# model.load_state_dict(checkpoint['model_state_dict'])
# avg_model = hydra.utils.instantiate(cfg.averaging, model=model)
# avg_model.load_state_dict(checkpoint['avg_model_state_dict'])
    

17:20:51 - INFO - Using device: cuda
17:20:57 - INFO - Found checkpoint: /orcd/data/omarabu/001/njwfish/counting_flows/outputs/84cddc73ed43/model.pt


Train split: 931, Test split: 49
Train cells: 1202750, Test cells: 61517


In [32]:
import numpy as np

idx = dataset.base_individual_idx
full_matrix_sum = np.zeros((idx.shape[0], len(dataset._eligible_cidx)), dtype=np.int32)
full_matrix_mean = np.zeros((idx.shape[0], len(dataset._eligible_cidx)), dtype=np.float32)
full_matrix_max = np.zeros((idx.shape[0], len(dataset._eligible_cidx)), dtype=np.int32)

for i, (ci, gs, ge) in enumerate(zip(dataset._eligible_cidx, dataset._eligible_start, dataset._eligible_end)):
    if i % 100 == 0:
        print(i)
    chrom_beg  = dataset.chrom_starts[ci]
    gene_start = int(chrom_beg + int(gs))
    gene_end   = int(chrom_beg + int(ge))  # end-exclusive
    gene_len   = int(gene_end - gene_start)

    # Precompute per-cell coverage over the whole gene once; we’ll slice per window
    # (shape: [n_cells, gene_len])
    x_1_global = dataset.fast_get_overlap_raw(idx, (gene_start, gene_end))
    full_matrix_sum[:, i] = x_1_global.sum(axis=1)
    full_matrix_mean[:, i] = x_1_global.mean(axis=1)
    full_matrix_max[:, i] = x_1_global.max(axis=1)

# save full_matrixes
np.save("full_matrix_sum.npy", full_matrix_sum)
np.save("full_matrix_mean.npy", full_matrix_mean)
np.save("full_matrix_max.npy", full_matrix_max)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [69]:
# load matrices
import numpy as np

full_matrix_sum = np.load("full_matrix_sum.npy")
full_matrix_mean = np.load("full_matrix_mean.npy")
full_matrix_max = np.load("full_matrix_max.npy")

In [70]:
full_matrix_max.var(axis=0).shape

(23963,)

In [71]:
sidx = np.argsort(full_matrix_max.var(axis=0))
gene_len = (dataset._eligible_end - dataset._eligible_start)
ensg_idx = dataset._eligible_gene_name.str.contains("ENSG")

max_len = 100_000
top_k_hvgs = 20



In [72]:
gene_len[sidx][(gene_len[sidx] < max_len) & ~ensg_idx[sidx]][-top_k_hvgs:]

array([ 7814,  7508, 21619,  2396,  5077, 21570,  4522,  7105,  5127,
        7098,  2784,  9089, 37494, 36280, 26277,  6729, 28181,  4514,
        4671,   523])

In [73]:
dataset._eligible_gene_name[sidx]

12755    ENSG00000254680
20009    ENSG00000267249
20013          CEP192-DT
20015    ENSG00000267177
12741             ADM-DT
              ...       
15793              RPS29
16176               IGHM
16171              IGHA1
3188                IGKC
3187     ENSG00000240040
Name: gene_name, Length: 23963, dtype: object

In [75]:
import pickle as pkl
# save gene names to pickle
pkl.dump(dataset._eligible_gene_name[sidx][(gene_len[sidx] < max_len) & ~ensg_idx[sidx]][-top_k_hvgs:].values, open("hvg_gene_names.pkl", "wb"))

In [77]:
dataset = hydra.utils.instantiate(cfg.dataset, hvg_only=True)

Train split: 931, Test split: 49
Train cells: 1202750, Test cells: 61517


In [78]:
import numpy as np

idx = np.concatenate(list(dataset.test_individual_idxs.values()))
full_matrix_sum = np.zeros((idx.shape[0], len(dataset._eligible_cidx)), dtype=np.int32)
full_matrix_mean = np.zeros((idx.shape[0], len(dataset._eligible_cidx)), dtype=np.float32)
full_matrix_max = np.zeros((idx.shape[0], len(dataset._eligible_cidx)), dtype=np.int32)

for i, (ci, gs, ge) in enumerate(zip(dataset._eligible_cidx, dataset._eligible_start, dataset._eligible_end)):
    # if i % 10 == 0:
    print(i)
    # if i < 44:
    #    continue
    chrom_beg  = dataset.chrom_starts[ci]
    gene_start = int(chrom_beg + int(gs))
    gene_end   = int(chrom_beg + int(ge))  # end-exclusive
    gene_len   = int(gene_end - gene_start)
    print(gene_len)

    # Precompute per-cell coverage over the whole gene once; we’ll slice per window
    # (shape: [n_cells, gene_len])
    x_1_global = dataset.fast_get_overlap_raw(idx, (gene_start, gene_end))
    g_sum = x_1_global.sum(axis=1)
    g_max = x_1_global.max(axis=1)
    full_matrix_sum[:, i] += g_sum
    print(full_matrix_max[:, i].sum(), g_max.sum())
    full_matrix_max[:, i] = np.maximum(full_matrix_max[:, i], g_max)
    print(full_matrix_max[:, i].sum(), g_max.sum())
        
    full_matrix_mean[:, i] = full_matrix_sum[:, i] / gene_len

# save full_matrixes
# np.save("test_full_matrix_sum.npy", full_matrix_sum)
# np.save("test_full_matrix_mean.npy", full_matrix_mean)
# np.save("test_full_matrix_max.npy", full_matrix_max)

0
523
0 307522
307522 307522
1
7098
0 1052771
1052771 1052771
2
26277


0 36573
36573 36573
3
5077
0 788398
788398 788398
4
7105
0 1002400
1002400 1002400
5
36280
0 1442864
1442864 1442864
6
37494
0 471496
471496 471496
7
2396
0 781199
781199 781199
8
21619
0 784679
784679 784679
9
4522
0 905614
905614 905614
10
7814
0 700197
700197 700197
11
28181
0 1756021
1756021 1756021
12
4671
0 106737
106737 106737
13
6729
0 17108
17108 17108
14
4514
0 51750
51750 51750
15
2784
0 923374
923374 923374
16
9089
0 1086214
1086214 1086214
17
7508
0 794819
794819 794819
18
21570
0 762126
762126 762126
19
5127
0 824009
824009 824009


In [79]:
np.save("test_full_matrix_sum.npy", full_matrix_sum)
np.save("test_full_matrix_mean.npy", full_matrix_mean)
np.save("test_full_matrix_max.npy", full_matrix_max)

In [81]:
np.save("test_target_cond.npy", dataset.target_cond[idx])

In [101]:
cum_idx = 0
test_mat_idxs = {}
for i, idxs in enumerate(dataset.test_individual_idxs.values()):
    test_mat_idxs[i] = np.arange(cum_idx, cum_idx + idxs.shape[0])
    cum_idx += len(idxs)
np.save("test_mat_idxs.npy", test_mat_idxs)
